In [56]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import dailydata
import requests
import collections
import os

from sklearn import linear_model
import statsmodels.api as sm

from backtesting import Backtest

from scipy.optimize import minimize


SyntaxError: invalid syntax (<ipython-input-56-315f6f0df9c9>, line 17)

In [2]:
with open("words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]
# words = words[:5] + ["debt"]
#words = words[:10]

word_trends = {}

In [3]:
x = os.walk("trends")
for word in x:
    tuplee = word
words = tuplee[2]

In [15]:
def getStats(key_word, start_year, start_month, end_year, end_month, index):
    #trend = pd.DataFrame(dailydata.get_daily_data(key_word, start_year, start_month, end_year, end_month, wait_time = 0)[key_word])
    trend = pd.read_csv("trends/"+key_word)
    key_word = key_word[:-4]
    trend['date'] = pd.to_datetime(trend['date'])
    trend = trend.set_index('date')
    trend["change"] = trend[key_word].pct_change()

    # grab index data
    index_data = yf.download(index, start = str(start_year)+"-"+str(start_month)+"-01", end = str(end_year)+"-"+str(end_month)+"-01")

    # join trend data with DJIA data
    joined = trend.merge(index_data, left_on = trend.index, right_on = index_data.index)
    joined = joined.rename(columns = {"key_0": "Date"})

    # grab the adj close price difference for each day
    #joined['Diff'] = joined['Adj Close'].diff()
    joined = joined.set_index(joined['Date'])

    joined["change moving avg"] = joined["change"].rolling("14d", min_periods = 1).mean()
    joined = joined[joined.index.dayofweek == 1]
    
    return joined


In [16]:
start_year = 2010
start_month = 1
end_year = 2020
end_month = 11
index = "NDAQ"

#for word in words:
for word in words:
    if word[0] == ".": # extra thing in there
        continue
    word_trends[word] = getStats(word, start_year, start_month, end_year, end_month, index)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [17]:
def ourFunc(df):
    return pd.Series(df['change'])

def ourFunc1(df):
    return pd.Series(df['change moving avg'])

In [51]:
from backtesting import Strategy
from backtesting.lib import crossover


class ourStrat(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    #n1 = 10
    #n2 = 20
    threshold = 0 # for debt, around 
    
    
    def init(self):
        # Precompute the two moving averages
        #self.sma1 = self.I(SMA, self.data.Close, self.n1)
        #self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.change = self.I(ourFunc1, self.data)
        self.own = False
        threshold = self.threshold # 0.05145242719080212 for debt
        #self.buy_from_last_week = False
        #self.sell_from_last_week = True
    
    def next(self):
        """
        if self.buy_from_last_week:
            self.buy()
            self.own = True
            self.buy_from_last_week = False
        
        elif self.sell_from_last_week:
            self.sell()
            self.own = False
            self.sell_from_last_week = False
        """
        
        if self.change[-1] > self.threshold: #and not self.own: #crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
            #self.own = True

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif self.change[-1] < -self.threshold: #and self.own: #crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            #self.own = False

In [60]:
returns = {}
sharpe = {}
win_rate = {}
avg_trade = {}

for word in word_trends.keys():
    bt = Backtest(word_trends[word], ourStrat, cash=100_000_000, commission=0)
    """
    stats = bt.optimize(threshold = list(np.asarray(range(-100, 100))/100), maximize = 'Return [%]', 
                       return_optimization = True, method = 'skopt')
    """
    stats = bt.run()
    returns[word] = stats[6]
    returns['BENCHMARK'] = stats[7]
    sharpe[word] = stats[10]
    win_rate[word] = stats[18]
    avg_trade[word] = stats[21]


dict_keys(['chance.csv', 'dividend.csv', 'hedge.csv', 'bonds.csv', 'banking.csv', 'derivatives.csv', 'sell.csv', 'office.csv', 'society.csv', 'crash.csv', 'money.csv', 'growth.csv', 'rich.csv', 'investment.csv', 'debt.csv', 'earnings.csv', 'present.csv', 'financial markets.csv', 'markets.csv', 'crisis.csv', 'inflation.csv', 'freedom.csv', 'gains.csv', 'world.csv', 'war.csv', 'unemployment.csv', 'return.csv', 'returns.csv'])

In [58]:
returns = {}
sharpe = {}
win_rate = {}
avg_trade = {}

count = 0
for word in word_trends.keys():
    if count > 0:
        break
    word = 'debt.csv'
    bt = Backtest(word_trends[word], ourStrat, cash=100_000_000, commission=0)
    stats = bt.optimize(threshold = list(np.asarray(range(-100, 100))/100), maximize = 'Return [%]', 
                       return_optimization = True, method = 'skopt')
    """
    returns[word] = stats[6]
    returns['BENCHMARK'] = stats[7]
    sharpe[word] = stats[10]
    win_rate[word] = stats[18]
    avg_trade[word] = stats[21]
    """
    count += 1
    
#returns
stats

(Start                     2010-01-05 00:00:00
 End                       2020-10-27 00:00:00
 Duration                   3948 days 00:00:00
 Exposure Time [%]                     99.6422
 Equity Final [$]                  7.26665e+08
 Equity Peak [$]                   7.83125e+08
 Return [%]                            626.665
 Buy & Hold Return [%]                 516.256
 Return (Ann.) [%]                     144.508
 Volatility (Ann.) [%]                 130.182
 Sharpe Ratio                          1.11005
 Sortino Ratio                         4.40279
 Calmar Ratio                          4.80628
 Max. Drawdown [%]                    -30.0666
 Avg. Drawdown [%]                     -5.9678
 Max. Drawdown Duration      315 days 00:00:00
 Avg. Drawdown Duration       66 days 00:00:00
 # Trades                                  463
 Win Rate [%]                          57.2354
 Best Trade [%]                         29.398
 Worst Trade [%]                      -19.2485
 Avg. Trade [

In [66]:
bt.plot()

### Random Alg

In [ ]:
import random

# randomly buy and sell 
class randomStrat(Strategy):
    def init(self):
        self.change = self.I(ourFunc1, self.data)
    def next(self):
        if random.choice([True, False]):
            self.position.close()
            self.buy()
        else:
            self.position.close()
            self.sell()

In [ ]:
random_returns = []
for x in range(201):
    bt = Backtest(word_trends["debt"], randomStrat, cash=100_000_000, commission=0) # initializes
    random_stats = bt.run()
    # returns["RANDOM"] = random_stats[6]
    random_returns.append(random_stats[6])

In [ ]:
np.mean(random_returns)

In [33]:
stats['Return [%]']

12.438247984760284